In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

import numpy as np

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
x_train = x_train.reshape(-1, 784) / 255.0  
x_test = x_test.reshape(-1, 784) / 255.0

In [5]:

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


In [6]:
len(y_train[0])

10

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))


Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0261 - accuracy: 0.8175 - val_loss: 0.0218 - val_accuracy: 0.8490
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0198 - accuracy: 0.8626 - val_loss: 0.0206 - val_accuracy: 0.8582
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0180 - accuracy: 0.8759 - val_loss: 0.0203 - val_accuracy: 0.8605
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0171 - accuracy: 0.8828 - val_loss: 0.0221 - val_accuracy: 0.8483
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0162 - accuracy: 0.8893 - val_loss: 0.0183 - val_accuracy: 0.8738
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0155 - accuracy: 0.8940 - val_loss: 0.0190 - val_accuracy: 0.8697
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0150 - accuracy: 0.8973 - val_loss: 0.0182 - val_accuracy:

In [9]:
loss = tf.keras.losses.MeanSquaredError()
opti = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.get_weights()

In [10]:
thresholds = [0.1 , 0.2, 0.3, 0.4, 0.5]
accu = []

In [11]:
    


for threshold in thresholds :

    num_epochs = 10
    for epoch in range(num_epochs):
        # pruning mask
        initial_weights = model.get_weights()
        pruning_mask = []
    
    
        for weights in model.get_weights() :
            mask = abs(weights) > threshold
            pruning_mask.append(mask.astype(np.float32))
    
        pruned_weights = [w*mask for w,mask in zip(initial_weights,pruning_mask)]
        model.set_weights(pruned_weights)
        
        with tf.GradientTape() as tape:
            # Forward pass
            outputs = model(x_train)
            loss_value = loss(outputs, y_train)
        
    
        # Backward pass and weight update
        gradients = tape.gradient(loss_value, model.trainable_variables)
        opti.apply_gradients(zip(gradients, model.trainable_variables))
    
    _ , acc = model.evaluate(x_test,y_test)
    accu.append(f'Threshold: {threshold}, Accuracy: {acc}')

accu





313/313 [==============================] - 1s 2ms/step - loss: 0.0900 - accuracy: 0.1000


['Threshold: 0.1, Accuracy: 0.8899000287055969',
 'Threshold: 0.2, Accuracy: 0.8245999813079834',
 'Threshold: 0.3, Accuracy: 0.2583000063896179',
 'Threshold: 0.4, Accuracy: 0.09619999676942825',
 'Threshold: 0.5, Accuracy: 0.10000000149011612']